In [ ]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, CSVLoader

In [ ]:
pdf_loader = PyPDFLoader("menu.pdf")
txt_loader = TextLoader("faq.txt")
csv_loader = CSVLoader("offers.csv")

docs = pdf_loader.load() + txt_loader.load() + csv_loader.load()
print(f"✅ Loaded {len(docs)} documents")

✅ Loaded 6 documents


In [ ]:
# ✅ Split and embed
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs_split = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs_split, embeddings)

C:\Users\ANSH KEDIA\AppData\Local\Temp\ipykernel_17868\1172452387.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\ANSH KEDIA\GenAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ✅ Load FLAN-T5 model
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
C:\Users\ANSH KEDIA\AppData\Local\Temp\ipykernel_17868\2282412579.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# ✅ Build retrieval chain
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
# ✅ Interact
query = "What are today's best offers?"
response = qa_chain.invoke(query)
print("💬", response["result"])

💬 Happy Hours,20% off on all drinks between 4 PM - 6 PM,2025-06-30 Offer Name,Description,Valid Till: Weekend Special,Buy 1 Get 1 Free on Cold Coffees (Sat & Sun),2025-07-31 Offer Name,Description,Valid Till: Student Discount,10% off with student ID,2025-12-31
